# Hi :)

----

In [1]:
import cv2
import numpy as np
from moviepy.video.VideoClip import VideoClip
from PIL import Image, ImageDraw

def image_to_video(image_path, output_video_path, scale_factor, duration):
    def detect_and_scale(image_path, scale_factor):
        # Load the image
        image= cv2.imread(image_path)
        gray= cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
        # Detect the black dot
        _, binary_dot= cv2.threshold(gray, 30, 255, cv2.THRESH_BINARY_INV)
        contours_dot, _= cv2.findContours(binary_dot, cv2.RETR_TREE, cv2.CHAIN_APPROX_SIMPLE)
        detected_dot= None
        for contour in contours_dot:
            area= cv2.contourArea(contour)
            if 10 < area < 500:  # Filter small regions
                M= cv2.moments(contour)
                if M["m00"] != 0:
                    cx= int(M["m10"] / M["m00"])
                    cy= int(M["m01"] / M["m00"])
                    detected_dot= (cx, cy)
                    break
        # Detect the outer edge
        _, binary_edge= cv2.threshold(gray, 200, 255, cv2.THRESH_BINARY_INV)
        contours_edge, hierarchy= cv2.findContours(binary_edge, cv2.RETR_TREE, cv2.CHAIN_APPROX_SIMPLE)
        outer_edge= None
        for i, contour in enumerate(contours_edge):
            if hierarchy[0][i][3] == -1:  # Outer edge has no parent
                outer_edge= contour
                break
        # Scale the outer edge
        scaled_outer_edge= []
        if outer_edge is not None:
            M= cv2.moments(outer_edge)
            cx= int(M["m10"] / M["m00"]) if M["m00"] != 0 else 0
            cy= int(M["m01"] / M["m00"]) if M["m00"] != 0 else 0
            center= np.array([cx, cy])
            for point in outer_edge:
                vector_to_center= point[0] - center
                scaled_point= center + vector_to_center * scale_factor
                scaled_outer_edge.append(scaled_point.astype(np.int32))
            scaled_outer_edge= np.array(scaled_outer_edge, dtype= np.int32)
        return scaled_outer_edge, detected_dot
    def animate_sequential_lines(image_path, output_path, scaled_path, duration):
        image= Image.open(image_path).convert("RGB")
        base_frame= np.array(image)
        def make_frame(t):
            frame= base_frame.copy()
            img= Image.fromarray(frame)
            draw= ImageDraw.Draw(img)
            time_per_line= duration / len(scaled_path)
            for i, point in enumerate(scaled_path[ : -1]):
                start_point= point
                end_point= scaled_path[i + 1]
                line_start_time= i * time_per_line
                line_end_time= (i + 1) * time_per_line
                if line_start_time <= t < line_end_time:
                    progress= (t - line_start_time) / time_per_line
                    x1, y1= start_point
                    x2, y2= end_point
                    current_x= int(x1 + (x2 - x1) * progress)
                    current_y= int(y1 + (y2 - y1) * progress)
                    draw.line((x1, y1, current_x, current_y), fill= "red", width= 4)
                elif t >= line_end_time:
                    x1, y1= start_point
                    x2, y2= end_point
                    draw.line((x1, y1, x2, y2), fill= "red", width= 5)
            return np.array(img)
        animation= VideoClip(make_frame, duration= duration)
        animation.write_videofile(output_path, fps= 24)
    # Detect and scale
    scaled_path, detected_dot= detect_and_scale(image_path, scale_factor)
    if detected_dot is None or len(scaled_path) == 0:
        raise ValueError("Failed to detect required features in the image.")
    # Transform the scaled path
    target= np.array([detected_dot[0], detected_dot[1]])
    distances= np.linalg.norm(scaled_path - target, axis= 1)
    closest_idx= np.argmin(distances)
    before_closest= scaled_path[:closest_idx]
    after_closest= scaled_path[closest_idx + 1:]
    result= np.vstack((scaled_path[closest_idx : closest_idx + 1], np.flip(before_closest, axis= 0), np.flip(after_closest, axis= 0)))    
    # Cut and append elements
    n= 4
    if len(result) > n:
        first_n= result[:n]
        remaining= result[n:]
        result= np.vstack((remaining, first_n))
    # Animate the lines
    animate_sequential_lines(image_path, output_video_path, result, duration)

# # Example usage
# input_image= "Circle.jpg"  # Replace with the path to your image
# output_video= "output_animation.mp4"
# image_to_video(input_image, output_video, scale_factor= 0.87, duration= 4)

In [7]:
input_image= "Star.jpg"  # Replace with the path to your image
output_video= "Star.mp4"
image_to_video(input_image, output_video, scale_factor= 0.9, duration= 4)

MoviePy - Building video Star.mp4.
MoviePy - Writing video Star.mp4



MoviePy - Done !
MoviePy - video ready Star.mp4
